## Assessment of Dataset and Database Management System
In this analysis, we perform quick analysis of our dataset and prototyping of which DBMS to use for our final project. We decided to use PostgreSQL because it is stable and reliable. We also studied two secure methods, SQLAlchemy and Psycopg2, that allow Python code to connect to PostgreSQL and effectively execute query commands in a database session.

Both SQLAlchemy and Psycopg2 provide secure and reliable connections with SQL databases, including PostgreSQL and MySQL, and are actively maintained. Our team decided to use:

- Dataset: Churn rate of customers' involvement history.
- DBMS: PostgreSQL.
- SQLAlchemy: SQL Toolkit and Object Relational Mapper (ORM), which provides a reliable and comprehensive set of tools for working with databases and Python.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.schema import CreateTable, DropTable, CreateColumn

# Import postgres pass
import os
from dotenv import load_dotenv
load_dotenv('../../.env')
pga_pass = os.getenv('PGA_PASS')

In [2]:
# Dependencies for ML
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.ensemble import AdaBoostClassifier

In [3]:
# Read the data into a Pandas DataFrame
df = pd.read_csv('./Resources/Churn_Modelling_2.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# drop unnecessary columns
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [5]:
# check null values
df.isna().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [6]:
# check number of duplicates
df.duplicated().sum()

0

In [7]:
# summary statistics of df
df.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [8]:
# Determine the number of unique values in each column.
df.nunique()

CreditScore         460
Geography             3
Gender                2
Age                  70
Tenure               11
Balance            6382
NumOfProducts         4
HasCrCard             2
IsActiveMember        2
EstimatedSalary    9999
Exited                2
dtype: int64

In [9]:
# Saving the clean_df to csv
file_path = "./Data/Churn_Modelling_clean.csv"
df.to_csv(file_path, index=False)

In [10]:
# Create our features (after dropping our target)
target = ['Exited']
X = df.drop(target, axis=1)
str_cols = X.select_dtypes(include=['object']).columns
# Binary encoding using Pandas (multiple object columns)
X = pd.get_dummies(X, columns=str_cols, dtype='int')

# Create our target
y = df[target]

In [11]:
# Check the balance of our target values
y.value_counts()

Exited
0         7963
1         2037
dtype: int64

In [12]:
# Set seed number to 1 for each model
seedn = 1
nen = 100  # n_estimators number
# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seedn)
y_train.value_counts()

Exited
0         5983
1         1517
dtype: int64

In [13]:
# Reusable func to get the balanced accuracy score, confusion matrix, imbalanced clf report
def sum_func(lm, Xtest=X_test, ytest=y_test):
    # Making predictions using the testing data.
    y_pred = lm.predict(Xtest)
    # Calculated the balanced accuracy score
    bacc_score = balanced_accuracy_score(ytest, y_pred)
    # Calculating the confusion matrix
    cm = confusion_matrix(ytest, y_pred)
    cm_df = pd.DataFrame(
        cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
    )
    # Summarizing the imbalanced classification report
    iclf_report = classification_report_imbalanced(ytest, y_pred)
    # return multiple variables as tuples
    return (bacc_score, cm, cm_df, iclf_report)

### Ensemble Learners: Balanced Random Forest Classifier

In [14]:
# Resample the training data with the BalancedRandomForestClassifier
brf_model = BalancedRandomForestClassifier(n_estimators=nen, random_state=seedn)
# Fitting the model
brf_model = brf_model.fit(X_train, y_train)

In [15]:
# Calculated the balanced accuracy score, confusion matrix, imbalanced clf report
bacc_score, cm, cm_df, iclf_report = sum_func(brf_model)
# Display the balanced accuracy score
print("Balanced Accuracy Score: %.6f" % bacc_score)

Balanced Accuracy Score: 0.784557


In [16]:
# Display the confusion matrix
display(cm, cm_df)

array([[1599,  381],
       [ 124,  396]], dtype=int64)

,Predicted 0,Predicted 1
Actual 0,1599,381
Actual 1,124,396


In [17]:
# Print the imbalanced classification report
print(iclf_report)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.93      0.81      0.76      0.86      0.78      0.62      1980
          1       0.51      0.76      0.81      0.61      0.78      0.61       520

avg / total       0.84      0.80      0.77      0.81      0.78      0.62      2500



In [18]:
# List the features sorted in descending order by feature importance
importances = brf_model.feature_importances_

# Display feature importance as a DataFrame
importance_df = pd.DataFrame({'feature': X.columns, 'importance': importances}).sort_values(by='importance', ascending=False)
importance_df.head

<bound method NDFrame.head of               feature  importance
1                 Age    0.248049
3             Balance    0.140375
7     EstimatedSalary    0.135423
0         CreditScore    0.133436
4       NumOfProducts    0.123429
2              Tenure    0.086837
6      IsActiveMember    0.039975
9   Geography_Germany    0.022629
5           HasCrCard    0.019882
8    Geography_France    0.013942
11      Gender_Female    0.013428
10    Geography_Spain    0.011539
12        Gender_Male    0.011057>

### Ensemble Learners: Easy Ensemble AdaBoost Classifier

In [19]:
# Train the EasyEnsembleClassifier
ee_model = EasyEnsembleClassifier(n_estimators=nen, random_state=seedn)
# Fitting the model
ee_model = ee_model.fit(X_train, y_train)

In [20]:
# Calculated the balanced accuracy score, confusion matrix, imbalanced clf report
bacc_score, cm, cm_df, iclf_report = sum_func(ee_model)
# Display the balanced accuracy score
print("Balanced Accuracy Score: %.6f" % bacc_score)

Balanced Accuracy Score: 0.778234


In [21]:
# Display the confusion matrix
display(cm, cm_df)

array([[1593,  387],
       [ 129,  391]], dtype=int64)

,Predicted 0,Predicted 1
Actual 0,1593,387
Actual 1,129,391


In [22]:
# Print the imbalanced classification report
print(iclf_report)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.93      0.80      0.75      0.86      0.78      0.61      1980
          1       0.50      0.75      0.80      0.60      0.78      0.60       520

avg / total       0.84      0.79      0.76      0.81      0.78      0.61      2500



### Ensemble Learners: AdaBoost Classifier

In [23]:
# Train the AdaBoostClassifier
ab_model = AdaBoostClassifier(n_estimators=nen, random_state=seedn)
# Fitting the model
ab_model = ab_model.fit(X, y)

In [24]:
# Calculated the balanced accuracy score, confusion matrix, imbalanced clf report
bacc_score, cm, cm_df, iclf_report = sum_func(ab_model)
# Display the balanced accuracy score
print("Balanced Accuracy Score: %.6f" % bacc_score)

Balanced Accuracy Score: 0.729186


In [25]:
# Display the confusion matrix
display(cm, cm_df)

array([[1909,   71],
       [ 263,  257]], dtype=int64)

,Predicted 0,Predicted 1
Actual 0,1909,71
Actual 1,263,257


In [26]:
# Print the imbalanced classification report
print(iclf_report)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.88      0.96      0.49      0.92      0.69      0.50      1980
          1       0.78      0.49      0.96      0.61      0.69      0.45       520

avg / total       0.86      0.87      0.59      0.85      0.69      0.49      2500



### PostgreSQL Database Management from/to Python

In [27]:
# SQLAlchemy: Establish connection to postgresql
username='postgres'
pwd=pga_pass
dbase='churnmodel_db'
hostname='localhost'
port_id=5432
engine = create_engine(f"postgresql://{username}:{pwd}@{hostname}:{port_id}/{dbase}")

try:
    engine.connect()
    print(f'Connection to PostgreSQL db called {dbase} is successfully established!\n')
except Exception as error:
    print(error)

Connection to PostgreSQL db called churnmodel_db is successfully established!



In [28]:
# Create tables called clean_df in postgresql (PS: remove if_exists)
df.to_sql(name='clean_df', con=engine, index=False, if_exists='replace')

1000

In [29]:
# Retrieve data from postgresql
pd.read_sql("SELECT * FROM clean_df;", con=engine)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [30]:
# single column query results
query = 'SELECT "CreditScore" FROM clean_df;'
scores = engine.execute(query)
pd.DataFrame(scores).head(10)

,CreditScore
0,619
1,608
2,502
3,699
4,850
5,645
6,822
7,376
8,501
9,684


In [31]:
age_limit = 70
query = f'SELECT * FROM clean_df WHERE "Age" > {age_limit} AND "Exited" = 1;'
ages = engine.execute(query)
pd.DataFrame(ages)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,523,France,Female,73,7,0.00,2,0,0,130883.90,1
1,727,Spain,Male,71,8,0.00,1,1,1,198446.91,1
2,546,Germany,Female,74,8,114888.74,2,1,1,66732.63,1
3,779,France,Male,71,3,0.00,2,1,1,146895.36,1
4,408,France,Female,84,8,87873.39,1,0,0,188484.52,1
5,667,Spain,Male,71,4,137260.78,1,0,1,94433.08,1
6,576,Germany,Female,71,6,140273.47,1,1,1,193135.25,1
7,624,France,Female,71,4,170252.05,3,1,1,73679.59,1
8,609,Germany,Female,71,6,113317.10,1,1,0,108258.22,1
9,782,Germany,Female,72,5,148666.99,1,1,0,2605.65,1


In [32]:
# group by Age and order by Exited counts
query = 'SELECT "Age", "Exited", COUNT("Exited") FROM clean_df WHERE "Exited" = 1 GROUP BY "Age", "Exited" ORDER BY COUNT DESC;'
ages = engine.execute(query)
pd.DataFrame(ages)

,Age,Exited,count
0,46,1,91
1,40,1,89
2,43,1,88
3,45,1,87
4,48,1,80
5,47,1,77
6,42,1,77
7,49,1,75
8,44,1,73
9,39,1,72


In [33]:
# Drop tables called stations and ridership in postgresql
# engine.execute("DROP TABLE clean_df CASCADE;")